# Build a simple LLM application with chat models and prompt templates

In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After reading this tutorial, you'll have a high level overview of:

Using language models

Using prompt templates

Debugging and tracing your application using LangSmith

Let's dive in!

## Installation

In [1]:
pip install langchain

Note: you may need to restart the kernel to use updated packages.


## LangSmith

Many of the applications you build with LangChain will contain multiple steps with multiple invocations of LLM calls. As these applications get more and more complex, it becomes crucial to be able to inspect what exactly is going on inside your chain or agent. The best way to do this is with LangSmith.



In [5]:
import getpass
import os

try:
    # load environment variables from .env file (requires `python-dotenv`)
    from dotenv import load_dotenv

    load_dotenv()
except ImportError:
    pass

os.environ["LANGSMITH_TRACING"] = "true"
if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass(
        prompt="Enter your LangSmith API key (optional): "
    )
if "LANGSMITH_PROJECT" not in os.environ:
    os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
        prompt='Enter your LangSmith Project Name (default = "default"): '
    )
    if not os.environ.get("LANGSMITH_PROJECT"):
        os.environ["LANGSMITH_PROJECT"] = "default"

Enter your LangSmith API key (optional):  ········
Enter your LangSmith Project Name (default = "default"):  ········


## Using Language Models

In [9]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

Enter API key for Google Gemini:  ········


Let's first use the model directly. ChatModels are instances of LangChain Runnables, which means they expose a standard interface for interacting with them. To simply call the model, we can pass in a list of messages to the .invoke method.

In [10]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

model.invoke(messages)

AIMessage(content='Ciao!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--ae9e74e5-6deb-4520-badb-e8f67cc2e9b2-0', usage_metadata={'input_tokens': 10, 'output_tokens': 36, 'total_tokens': 46, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 34}})

Note that ChatModels receive message objects as input and generate message objects as output. In addition to text content, message objects convey conversational roles and hold important data, such as tool calls and token usage counts.

LangChain also supports chat model inputs via strings or OpenAI format. The following are equivalent:

In [11]:
model.invoke("Hello")

model.invoke([{"role": "user", "content": "Hello"}])

model.invoke([HumanMessage("Hello")])

AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--f24e66e1-c65b-4b3d-b924-1905afcac686-0', usage_metadata={'input_tokens': 2, 'output_tokens': 37, 'total_tokens': 39, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 28}})

## Streaming

Because chat models are Runnables, they expose a standard interface that includes async and streaming modes of invocation. This allows us to stream individual tokens from a chat model:

In [12]:
for token in model.stream(messages):
    print(token.content, end="|")

Ciao!|

## Prompt Templates
Right now we are passing a list of messages directly into the language model. Where does this list of messages come from? Usually, it is constructed from a combination of user input and application logic. This application logic usually takes the raw user input and transforms it into a list of messages ready to pass to the language model. Common transformations include adding a system message or formatting a template with the user input.

Prompt templates are a concept in LangChain designed to assist with this transformation. They take in raw user input and return data (a prompt) that is ready to pass into a language model.

Let's create a prompt template here. It will take in two user variables:

language: The language to translate text into
text: The text to translate

In [13]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

Note that ChatPromptTemplate supports multiple message roles in a single template. We format the language parameter into the system message, and the user text into a user message.

The input to this prompt template is a dictionary. We can play around with this prompt template by itself to see what it does by itself

In [1]:
prompt = prompt_template.invoke({"language": "Telugu", "text": "Thank you"})

prompt

NameError: name 'prompt_template' is not defined

We can see that it returns a ChatPromptValue that consists of two messages. If we want to access the messages directly we do:

In [26]:
prompt.to_messages()

[SystemMessage(content='Translate the following from English into Japanese', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Thank you', additional_kwargs={}, response_metadata={})]

Finally, we can invoke the chat model on the formatted prompt:



In [31]:
response = model.invoke(prompt)
print(response.content)

ధన్యవాదాలు (Dhanyavādālu)


If we take a look at the LangSmith trace, we can see exactly what prompt the chat model receives, along with token usage information, latency, standard model parameters (such as temperature), and other information.



## Conclusion

That's it! In this tutorial you've learned how to create your first simple LLM application. You've learned how to work with language models, how to create a prompt template, and how to get great observability into applications you create with LangSmith.

This just scratches the surface of what you will want to learn to become a proficient AI Engineer. Luckily - we've got a lot of other resources!